Chargement des librairies

In [ ]:
import numpy as np
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

Chargement des données

In [ ]:
train_dir = 'chemin_vers_les_images_dentrainement'
val_dir = 'chemin_vers_les_images_de_validation'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224,224), batch_size=32, class_mode='categorical')

Chargement du modèle

In [ ]:
# Chargement de VGG16 pré-entraîné
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congélation des couches de VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Ajout des couches personnalisées pour la classification
x = vgg16.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(11, activation='softmax')(x)

# Compilation du modèle
model = Model(inputs=vgg16.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

Entraînement

In [ ]:
# Ajout des callbacks EarlyStopping et ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='max')
checkpoint = ModelCheckpoint('vgg16_best_model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Entraînement du modèle
history = model.fit(train_generator, steps_per_epoch=train_generator.samples//32, epochs=100, validation_data=val_generator, validation_steps=val_generator.samples//32, callbacks=[early_stopping, checkpoint])

Tracé des courbes

In [ ]:
# Tracé de l'accuracy et de la loss en fonction des epochs
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('accuracy_ vgg16.png')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('loss_vgg16.png')
plt.show()